In [1]:
#!/usr/bin/env python
import pygame
from __future__ import print_function
import os
os.environ["CUDA_VISIVBLE_DEVICES"] = '0'
import tensorflow as tf
import cv2
import sys
sys.path.append("game/")
import wrapped_flappy_bird as game
import random
import numpy as np
from collections import deque
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
print('tensorflow version: ',tf.__version__)


pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 3037498224429934671
]
tensorflow version:  1.10.0


In [2]:
import pygame
from __future__ import print_function
import numpy as np
from collections import deque
import cv2

In [3]:
GAME = 'bird' # the name of the game being played for log files
ACTIONS = 2 # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVE = 10000. # timesteps to observe before training
EXPLORE = 3000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.0001 # final value of epsilon
INITIAL_EPSILON = 0.5 # starting value of epsilon
REPLAY_MEMORY = 50000 # number of previous transitions to remember
BATCH = 32 # size of minibatch
FRAME_PER_ACTION = 1

In [4]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev = 0.01)
    return tf.Variable(initial)

In [5]:
def bias_variable(shape):
    initial = tf.constant(0.01, shape = shape)
    return tf.Variable(initial)

In [6]:
def conv2d(x, W, stride):
    return tf.nn.conv2d(x, W, strides = [1, stride, stride, 1], padding = "SAME")

In [7]:
def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = "SAME")

In [8]:
def createNetwork():
    # network weights
    W_conv1 = weight_variable([8, 8, 4, 32])
    b_conv1 = bias_variable([32])

    W_conv2 = weight_variable([4, 4, 32, 64])
    b_conv2 = bias_variable([64])

    W_conv3 = weight_variable([3, 3, 64, 64])
    b_conv3 = bias_variable([64])

    W_fc1 = weight_variable([1600, 512])
    b_fc1 = bias_variable([512])

    W_fc2 = weight_variable([512, ACTIONS])
    b_fc2 = bias_variable([ACTIONS])

    # input layer
    s = tf.placeholder("float", [None, 80, 80, 4])

    # hidden layers
    h_conv1 = tf.nn.relu(conv2d(s, W_conv1, 4) + b_conv1)
    h_pool1 = max_pool_2x2(h_conv1)

    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2, 2) + b_conv2)
    #h_pool2 = max_pool_2x2(h_conv2)

    h_conv3 = tf.nn.relu(conv2d(h_conv2, W_conv3, 1) + b_conv3)
    #h_pool3 = max_pool_2x2(h_conv3)

    #h_pool3_flat = tf.reshape(h_pool3, [-1, 256])
    h_conv3_flat = tf.reshape(h_conv3, [-1, 1600])

    h_fc1 = tf.nn.relu(tf.matmul(h_conv3_flat, W_fc1) + b_fc1)

    # readout layer
    readout = tf.matmul(h_fc1, W_fc2) + b_fc2

    return s, readout, h_fc1

In [9]:
def trainNetwork(s, readout, h_fc1, sess):
    # define the cost function
    a = tf.placeholder("float", [None, ACTIONS])
    y = tf.placeholder("float", [None])
    readout_action = tf.reduce_sum(tf.multiply(readout, a), reduction_indices=1)
    cost = tf.reduce_mean(tf.square(y - readout_action))
    train_step = tf.train.AdamOptimizer(1e-6).minimize(cost)

    # open up a game state to communicate with emulator
    game_state = game.GameState()

    # store the previous observations in replay memory
    D = deque()

    # printing
    a_file = open("logs_" + GAME + "/readout.txt", 'w')
    h_file = open("logs_" + GAME + "/hidden.txt", 'w')

    # get the first state by doing nothing and preprocess the image to 80x80x4
    do_nothing = np.zeros(ACTIONS)
    do_nothing[0] = 1
    x_t, r_0, terminal = game_state.frame_step(do_nothing)
    x_t = cv2.cvtColor(cv2.resize(x_t, (80, 80)), cv2.COLOR_BGR2GRAY)
    etr, x_t = cv2.threshold(x_t,1,255,cv2.THRESH_BINARY)
    s_t = np.stack((x_t, x_t, x_t, x_t), axis=2)

    # saving and loading networks
    saver = tf.train.Saver()
    sess.run(tf.initialize_all_variables())
    checkpoint = tf.train.get_checkpoint_state("saved_networks")
    if checkpoint and checkpoint.model_checkpoint_path:
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print("Successfully loaded:", checkpoint.model_checkpoint_path)
    else:
        print("Could not find old network weights")

    # start training
    epsilon = INITIAL_EPSILON
    t = 0 
    
    while "flappy bird" != "angry bird":
        # choose an action epsilon greedily
        readout_t = readout.eval(feed_dict={s : [s_t]})[0]
        a_t = np.zeros([ACTIONS])
        action_index = 0
        if t % FRAME_PER_ACTION == 0:
            if random.random() <= epsilon:
                print("----------Random Action----------")
                
                if random.random() < 0.1 :
                    a_t[1] = 1
                    action_index = 1
                else:
                    a_t[0] = 1
                    action_index = 0
            else:
                action_index = np.argmax(readout_t)
                a_t[action_index] = 1
        else:
            a_t[0] = 1 # do nothing

        # scale down epsilon
        if epsilon > FINAL_EPSILON and t > OBSERVE:
            epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE

        # run the selected action and observe next state and reward
        x_t1_colored, r_t, terminal = game_state.frame_step(a_t)
        x_t1 = cv2.cvtColor(cv2.resize(x_t1_colored, (80, 80)), cv2.COLOR_BGR2GRAY)
        ret, x_t1 = cv2.threshold(x_t1, 1, 255, cv2.THRESH_BINARY)
        x_t1 = np.reshape(x_t1, (80, 80, 1))
        #s_t1 = np.append(x_t1, s_t[:,:,1:], axis = 2)
        s_t1 = np.append(x_t1, s_t[:, :, :3], axis=2)

        # store the transition in D
        D.append((s_t, a_t, r_t, s_t1, terminal))
        if len(D) > REPLAY_MEMORY:
            D.popleft()

        # only train if done observing
        if t > OBSERVE:
            # sample a minibatch to train on
            minibatch = random.sample(D, BATCH)

            # get the batch variables
            s_j_batch = [d[0] for d in minibatch]
            a_batch = [d[1] for d in minibatch]
            r_batch = [d[2] for d in minibatch]
            s_j1_batch = [d[3] for d in minibatch]

            y_batch = []
            readout_j1_batch = readout.eval(feed_dict = {s : s_j1_batch})
            for i in range(0, len(minibatch)):
                terminal = minibatch[i][4]
                # if terminal, only equals reward
                if terminal:
                    y_batch.append(r_batch[i])
                else:
                    y_batch.append(r_batch[i] + GAMMA * np.max(readout_j1_batch[i]))

            # perform gradient step
            train_step.run(feed_dict = {
                y : y_batch,
                a : a_batch,
                s : s_j_batch}
            )

        # update the old values
        s_t = s_t1
        t += 1

        # save progress every 10000 iterations
        if t % 10000 == 0:
            saver.save(sess, 'saved_networks/' + GAME + '-dqn', global_step = t)

        # print info
        state = ""
        if t <= OBSERVE:
            state = "observe"
        elif t > OBSERVE and t <= OBSERVE + EXPLORE:
            state = "explore"
        else:
            state = "train"

        print("TIMESTEP", t, "/ STATE", state, \
            "/ EPSILON", epsilon, "/ ACTION", action_index, "/ REWARD", r_t, \
            "/ Q_MAX %e" % np.max(readout_t))
        # write info to files
        '''
        if t % 10000 <= 100:
            a_file.write(",".join([str(x) for x in readout_t]) + '\n')
            h_file.write(",".join([str(x) for x in h_fc1.eval(feed_dict={s:[s_t]})[0]]) + '\n')
            cv2.imwrite("logs_tetris/frame" + str(t) + ".png", x_t1)
        '''

In [ ]:
def playGame():
    sess = tf.InteractiveSession()
    s, readout, h_fc1 = createNetwork()
    trainNetwork(s, readout, h_fc1, sess)

def main():
    playGame()

if __name__ == "__main__":
    main()

Instructions for updating:
Use `tf.global_variables_initializer` instead.
INFO:tensorflow:Restoring parameters from saved_networks\bird-dqn-10000
Successfully loaded: saved_networks\bird-dqn-10000
TIMESTEP 1 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.215087e+01
----------Random Action----------
TIMESTEP 2 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.204418e+01
TIMESTEP 3 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.191034e+01
----------Random Action----------
TIMESTEP 4 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.184067e+01
----------Random Action----------
TIMESTEP 5 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.187737e+01
----------Random Action----------
TIMESTEP 6 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.189903e+01
----------Random Action----------
TIMESTEP 7 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.182255e+01
TIMESTEP 8 / STATE observe / EP

TIMESTEP 80 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.207721e+01
----------Random Action----------
TIMESTEP 81 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.224220e+01
----------Random Action----------
TIMESTEP 82 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.229432e+01
TIMESTEP 83 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.222490e+01
TIMESTEP 84 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.227765e+01
----------Random Action----------
TIMESTEP 85 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.232348e+01
----------Random Action----------
TIMESTEP 86 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.236263e+01
----------Random Action----------
TIMESTEP 87 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.225242e+01
TIMESTEP 88 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.225109e+01
TIMESTEP 89 / STATE observe / EPSILON 0.5 / ACT

TIMESTEP 160 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.189099e+01
TIMESTEP 161 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.189276e+01
----------Random Action----------
TIMESTEP 162 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.191957e+01
----------Random Action----------
TIMESTEP 163 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.192028e+01
TIMESTEP 164 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.197691e+01
TIMESTEP 165 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.195647e+01
TIMESTEP 166 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.196325e+01
TIMESTEP 167 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.195385e+01
----------Random Action----------
TIMESTEP 168 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.192622e+01
TIMESTEP 169 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.201404e+01
TIMESTEP 170 / STA

TIMESTEP 242 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.231382e+01
TIMESTEP 243 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.238040e+01
----------Random Action----------
TIMESTEP 244 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.237913e+01
TIMESTEP 245 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.223048e+01
----------Random Action----------
TIMESTEP 246 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.222825e+01
----------Random Action----------
TIMESTEP 247 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.234337e+01
----------Random Action----------
TIMESTEP 248 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.235933e+01
TIMESTEP 249 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.229477e+01
----------Random Action----------
TIMESTEP 250 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.248460e+01
----------Random Action----------
TIME

TIMESTEP 324 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.228464e+01
----------Random Action----------
TIMESTEP 325 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.236585e+01
TIMESTEP 326 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.229594e+01
----------Random Action----------
TIMESTEP 327 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.226132e+01
----------Random Action----------
TIMESTEP 328 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.225210e+01
TIMESTEP 329 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.227143e+01
----------Random Action----------
TIMESTEP 330 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.225937e+01
TIMESTEP 331 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.219966e+01
----------Random Action----------
TIMESTEP 332 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.220248e+01
TIMESTEP 333 / STATE observe / EPSILON

...

TIMESTEP 9690 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.191957e+01
----------Random Action----------
TIMESTEP 9691 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.192028e+01
----------Random Action----------
TIMESTEP 9692 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.197691e+01
----------Random Action----------
TIMESTEP 9693 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.197813e+01
----------Random Action----------
TIMESTEP 9694 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.192845e+01
----------Random Action----------
TIMESTEP 9695 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.191605e+01
TIMESTEP 9696 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.189286e+01
TIMESTEP 9697 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.187319e+01
TIMESTEP 9698 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.205917e+01
----------Random Action------

TIMESTEP 9770 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.196556e+01
----------Random Action----------
TIMESTEP 9771 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.207006e+01
TIMESTEP 9772 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.202684e+01
----------Random Action----------
TIMESTEP 9773 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.217307e+01
----------Random Action----------
TIMESTEP 9774 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.224375e+01
----------Random Action----------
TIMESTEP 9775 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.217628e+01
TIMESTEP 9776 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.227366e+01
----------Random Action----------
TIMESTEP 9777 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.228428e+01
----------Random Action----------
TIMESTEP 9778 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.227064

TIMESTEP 9848 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.253795e+01
TIMESTEP 9849 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.245920e+01
TIMESTEP 9850 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.247231e+01
TIMESTEP 9851 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.241191e+01
----------Random Action----------
TIMESTEP 9852 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.243776e+01
----------Random Action----------
TIMESTEP 9853 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.241368e+01
----------Random Action----------
TIMESTEP 9854 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.247729e+01
----------Random Action----------
TIMESTEP 9855 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.250105e+01
TIMESTEP 9856 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.257828e+01
TIMESTEP 9857 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD

TIMESTEP 9929 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.259052e+01
----------Random Action----------
TIMESTEP 9930 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.262229e+01
----------Random Action----------
TIMESTEP 9931 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.274730e+01
----------Random Action----------
TIMESTEP 9932 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.269903e+01
TIMESTEP 9933 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.262952e+01
TIMESTEP 9934 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.265536e+01
----------Random Action----------
TIMESTEP 9935 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.268340e+01
TIMESTEP 9936 / STATE observe / EPSILON 0.5 / ACTION 0 / REWARD 0.1 / Q_MAX 1.268543e+01
----------Random Action----------
TIMESTEP 9937 / STATE observe / EPSILON 0.5 / ACTION 1 / REWARD 0.1 / Q_MAX 1.264328e+01
----------Random Action------

TIMESTEP 10007 / STATE explore / EPSILON 0.4999990001999999 / ACTION 0 / REWARD 0.1 / Q_MAX 1.222924e+01
----------Random Action----------
TIMESTEP 10008 / STATE explore / EPSILON 0.4999988335666665 / ACTION 0 / REWARD 0.1 / Q_MAX 1.218882e+01
----------Random Action----------
TIMESTEP 10009 / STATE explore / EPSILON 0.49999866693333317 / ACTION 0 / REWARD 0.1 / Q_MAX 1.222443e+01
----------Random Action----------
TIMESTEP 10010 / STATE explore / EPSILON 0.4999985002999998 / ACTION 0 / REWARD 0.1 / Q_MAX 1.222479e+01
----------Random Action----------
TIMESTEP 10011 / STATE explore / EPSILON 0.49999833366666646 / ACTION 0 / REWARD 0.1 / Q_MAX 1.227346e+01
----------Random Action----------
